In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import cdist
from skimage import measure
from matplotlib import pyplot as plt
import os
import tifffile
from tqdm import tqdm
from typing import List, Tuple

In [2]:
selected_magnification = "20x"
selected_image_type = "dw"
max_dist = 9
aspect = np.array((1/3, 1/3))    # XY: 20x / aspect => 60x

### Read shift data

In [3]:
shifts = pd.read_csv(f"shift_correction/{selected_magnification}_{selected_image_type}.shifts.csv")
shifts.index = shifts["sid"].values
shifts.drop("sid", 1, inplace=True)

# Matching 20x_dw and reference dots

In [4]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")
dots_data = dots_data[selected_magnification == dots_data["magnification"]]
dots_data = dots_data[selected_image_type == dots_data["image_type"]]

In [5]:
thresholds_table = pd.read_csv("../../data/magnifications_matching/intensity_thresholds.by_field.tsv", sep="\t")

In [6]:
reference = pd.read_csv("../../data/60x_reference/ref__dw.field_thr.tsv", sep="\t")

In [7]:
def get_unassigned_count(pdist: np.ndarray) -> int:
    return np.sum(np.nansum(np.isnan(pdist), 1) != pdist.shape[1])

def match_dots(dotsA: np.ndarray, dotsB: np.ndarray) -> List[Tuple[int, int, float]]:
    pdist = cdist(dotsA, dotsB)
    
    matched: List[Tuple[int, int, float]] = []
    unassigned_rows = pdist.shape[0] + 1
    iteration_counter = 0
    print("Iteration #", end="")
    while unassigned_rows > get_unassigned_count(pdist) and 0 < get_unassigned_count(pdist):
        iteration_counter += 1
        print(f"...{iteration_counter}", end="")

        unassigned_rows = get_unassigned_count(pdist)
        A_id = 0
        while A_id < pdist.shape[0]:
            if pdist.shape[1] == np.sum(np.isnan(pdist[A_id, :])):
                A_id += 1
                continue
            B_id = np.nanargmin(pdist[A_id, :])
            if np.nanargmin(pdist[:, B_id]) == A_id:
                matched.append((A_id, B_id, pdist[A_id, B_id]))
                pdist[A_id, :] = np.nan
                pdist[:, B_id] = np.nan
            A_id += 1
    print("")
    return matched

In [8]:
match_output: List[pd.DataFrame] = []
match_output_full: List[pd.DataFrame] = []
match_counts: List[Tuple[int, int, int, int]] = []
for current_field_id in tqdm(np.unique(dots_data["sid"])):
    thresholds = thresholds_table.loc[current_field_id == thresholds_table["sid"], :]
    intensity_thr = thresholds.loc[selected_image_type == thresholds["image_type"], "thr"].values[0]

    dots = dots_data.loc[current_field_id == dots_data["sid"], :].copy(
        ).sort_values("Value2", ascending=False).reset_index(drop=True)
    
    dot_coords = dots.loc[intensity_thr <= dots["Value2"], ("x", "y")].copy().reset_index(drop=True)
    dot_coords2 = dot_coords.copy() / aspect
    dot_coords2["x"] += (shifts.loc[current_field_id, "x"] * 9)
    dot_coords2["y"] += (shifts.loc[current_field_id, "y"] * 9)
    ref_coords = reference.loc[reference["sid"] == current_field_id, ("x", "y")].copy().reset_index(drop=True)
    
    matched: List[Tuple[int, int, float]] = match_dots(dot_coords2, ref_coords)
    
    matched_a = np.array(matched)
    matched_field = pd.DataFrame(dict(
        series=current_field_id,
        id_20x=matched_a[:, 0].astype("i"),
        id_60x=matched_a[:, 1].astype("i"),
        eudist=matched_a[:, 2]
    ))
    matched_field_within_dist = matched_field[matched_field["eudist"] <= max_dist]
    match_output.append(matched_field_within_dist)
    match_output_full.append(matched_field)
    match_counts.append((current_field_id,
                         matched_field_within_dist.shape[0],
                         matched_field.shape[0]-matched_field_within_dist.shape[0],
                         dot_coords2.shape[0],
                         ref_coords.shape[0]))

  0%|          | 0/7 [00:00<?, ?it/s]

Iteration #...1...2...3...4...5...6...7...8...9...10...11...12...13

 14%|█▍        | 1/7 [00:00<00:03,  1.85it/s]

...14
Iteration #...1...2...3...4...5...6...7...8...9...10...11

 29%|██▊       | 2/7 [00:01<00:03,  1.61it/s]

...

 71%|███████▏  | 5/7 [00:03<00:01,  1.15it/s]


Iteration #...1...2...3...4...5...6...7

100%|██████████| 7/7 [00:04<00:00,  1.91it/s]

...8...9...10...11
Iteration #...1...2...3...4...5...6...7...8...9...10...11...12


100%|██████████| 7/7 [00:04<00:00,  1.62it/s]


In [9]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:, 1])
n_total_over_dist = np.sum(match_counts_a[:, 2])
n_total_20x = np.sum(match_counts_a[:, 3])
n_total_60x = np.sum(match_counts_a[:, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

7035/8734 (80.55%) 20x dots matched to 60x
7035/9944 (70.75%) 60x dots matched to 20x
1670 (19.12% 20x, 16.79% 60x) dots matched with distance > 9


In [10]:
for series_id in range(match_counts_a.shape[0]):
    n_matched = match_counts_a[series_id, 1]
    n_over_dist = match_counts_a[series_id, 2]
    n_20x = match_counts_a[series_id, 3]
    n_60x = match_counts_a[series_id, 4]
    print(f"""Field #{series_id+1}
    {n_matched}/{n_20x} ({n_matched/n_20x*100:.2f}%) 20x dots matched to 60x
    {n_matched}/{n_60x} ({n_matched/n_60x*100:.2f}%) 60x dots matched to 20x
    {n_over_dist} ({n_over_dist/n_20x*100:.2f}% 20x, {n_over_dist/n_60x*100:.2f}% 60x) dots matched with distance > {max_dist}
    """)

Field #1
    1023/1301 (78.63%) 20x dots matched to 60x
    1023/1272 (80.42%) 60x dots matched to 20x
    249 (19.14% 20x, 19.58% 60x) dots matched with distance > 9
    
Field #2
    1425/1731 (82.32%) 20x dots matched to 60x
    1425/1865 (76.41%) 60x dots matched to 20x
    306 (17.68% 20x, 16.41% 60x) dots matched with distance > 9
    
Field #3
    774/948 (81.65%) 20x dots matched to 60x
    774/1032 (75.00%) 60x dots matched to 20x
    174 (18.35% 20x, 16.86% 60x) dots matched with distance > 9
    
Field #4
    1253/1523 (82.27%) 20x dots matched to 60x
    1253/1534 (81.68%) 60x dots matched to 20x
    270 (17.73% 20x, 17.60% 60x) dots matched with distance > 9
    
Field #5
    1838/2261 (81.29%) 20x dots matched to 60x
    1838/2471 (74.38%) 60x dots matched to 20x
    423 (18.71% 20x, 17.12% 60x) dots matched with distance > 9
    
Field #6
    516/664 (77.71%) 20x dots matched to 60x
    516/869 (59.38%) 60x dots matched to 20x
    148 (22.29% 20x, 17.03% 60x) dots matche

#### General statistics, excluding field #7

In [11]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:6, 1])
n_total_over_dist = np.sum(match_counts_a[:6, 2])
n_total_20x = np.sum(match_counts_a[:6, 3])
n_total_60x = np.sum(match_counts_a[:6, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6829/8428 (81.03%) 20x dots matched to 60x
6829/9043 (75.52%) 60x dots matched to 20x
1570 (18.63% 20x, 17.36% 60x) dots matched with distance > 9


#### General statistics, excluding fields #6-7

In [12]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:5, 1])
n_total_over_dist = np.sum(match_counts_a[:5, 2])
n_total_20x = np.sum(match_counts_a[:5, 3])
n_total_60x = np.sum(match_counts_a[:5, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6313/7764 (81.31%) 20x dots matched to 60x
6313/8174 (77.23%) 60x dots matched to 20x
1422 (18.32% 20x, 17.40% 60x) dots matched with distance > 9


### Export

In [13]:
matched_dots = pd.concat(match_output)

In [14]:
matched_dots.to_csv(os.path.join("../../data/magnifications_matching",
                                 f"{selected_magnification}_{selected_image_type}.matched_dots.field_thr.tsv"
                                ), sep="\t", index=False)

In [15]:
pd.DataFrame(match_counts_a, columns=["series_id", "matched", "matched_over_dist", "raw_total", "dw_total"]
            ).to_csv(os.path.join("../../data/magnifications_matching",
                                  f"{selected_magnification}_{selected_image_type}.matched_dots.field_thr.counts.tsv"
                                 ), sep="\t", index=False)